In [10]:
import random
import simpy
import numpy as np
import scipy.stats as st


In [11]:
RANDOM_SEED = 42
TASK_MEAN = 24.0
# TASK_SIGMA = 2.0

INTERRUPTION_MEAN = 10.0
INTERRUPTION_SIGMA = 3.0

BREAK_MEAN = 6.0


random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [12]:
def time_per_task():
    return st.expon.rvs(size=1, scale=TASK_MEAN)[0]

def time_to_interrupt(enabled=1):
    if enabled:
        return st.expon.rvs(size=1, scale=INTERRUPTION_MEAN)[0]
    return 10000

def time_to_break(fixed=0):
    if fixed:
        return fixed
    return time_per_task()

def break_duration(fixed=0):
    if fixed:
        return fixed
    return st.expon.rvs(size=1, scale=BREAK_MEAN)[0]

def interruption_duration(enabled=1):
    if enabled:
        return st.expon.rvs(size=1, scale=INTERRUPTION_SIGMA)[0]
    return 0



In [13]:



class Person:
    def __init__(self,env,state, met=(0,0), interr=True):
        # Simpy Config
        self.state = state
        self.env = env
        self.person = simpy.PreemptiveResource(env,capacity=1)

        #Variables
        self.completed_tasks = 0
        self.breaks = 0
        self.interrupts = 0
        self.task_duration_sum = 0

        # Metodología
        self.b_time, self.b_duration= met
        self.interrupting_enabled = interr
        
        #Procesos
        self.process_working = env.process(self.working())
        env.process(self.interrupting())
        self.process_break = env.process(self.take_break())

    def working(self):
        while True:
            time = time_per_task()
            t_duration = time
            
            # print('duracion de la proxima tarea', time)
            it = ""
            
            while time:    
                start = self.env.now
                try:
                    print(f'Minuto {self.env.now} | {it}iniciando tarea {self.completed_tasks+1} | queda por trabajar {time} min')
                    self.state = 'W'

                    yield self.env.timeout(time)

                    # self.completed_tasks +=1
                    # print(f'Minuto {self.env.now} | tarea numero {self.completed_tasks} completada')
                    
                    time = 0
                    self.state = 'S'

                except simpy.Interrupt:
                    time = max(0, time + start - self.env.now)
                    
                    if self.state == 'I':
                        interruption_time = interruption_duration()
                        print(f'Minuto {self.env.now} | trabajo interrumpido por {interruption_time} | quedando por completar unos {time} min de tarea')
                        
                        yield self.env.timeout(interruption_time)
                        print(f'Minuto {self.env.now} | interrupción terminada')
                        self.interrupts += 1
                        #self.state = 'W'
                        
                    elif self.state == 'D': 
                        break_time = break_duration(self.b_duration)
                        start_break = self.env.now
                        print(f'Minuto {self.env.now} | descanso por {break_time} min | quedando por completar unos {time} min de tarea')

                        while break_time:
                            try:
                                yield self.env.timeout(break_time)
                                break_time = 0
                                #self.state = 'W'

                            except:
                                break_time = max(0, break_time + start_break - self.env.now)
                                interruption_time = interruption_duration()
                                print(f'Minuto {self.env.now} | descanso interrumpido por {interruption_time} min')
                                yield self.env.timeout(interruption_time)
                                print(f'Minuto {self.env.now} | interrupción terminada')
                                self.interrupts += 1
                                # yield self.env.timeout(break_time)
                                self.state = 'D'

                            self.breaks += 1
                            print(f'Minuto {self.env.now} | descanso completada en el minuto', self.env.now)
                                
                            

                    # yield self.env.timeout(time)
                    # time = 0
                    # self.state = 'S'            

                it='re'    
                
            self.completed_tasks += 1
            self.task_duration_sum += t_duration
            print(f'Minuto {self.env.now} | tarea numero {self.completed_tasks} completada')

    def take_break(self):
        dur = 0.0
        while True:
            time = time_to_break(self.b_time + dur)
            dur = self.b_duration
            # print('solicitando el proximo descanso en ',self.env.now, ' para dentro de ', time)
            yield self.env.timeout(time)
           
            with self.person.request(priority=1) as request:
                yield request
                    
                if self.state == 'W':
                    self.state = 'D'
                    self.process_working.interrupt()
                # elif self.state == 'I':
                #     print('no puedes descansar por interrupcion', self.env.now)
                

    def interrupting(self):
            while True:
                time = time_to_interrupt(self.interrupting_enabled)
                # print('solicitando proxima interrupcion en ', self.env.now, ' para dentro de :', time)
                yield self.env.timeout(time)
                
                with self.person.request(priority=0) as request:
                    yield request
                    # print('se obtuvo la interrupcion (el recurso) en el minuto', self.env.now)
                    if self.state == 'W':
                        self.state = 'I'
                        self.process_working.interrupt()
                    elif self.state == 'D':
                        self.state = 'I'
                        self.process_working.interrupt()
                    else:
                        print(f'no se puede interrumpir en el minuto', self.env.now)
                        continue
            


## Preguntas
- Cual es la media de la cantidad de tareas resultas siguiendo cada estrategia?

- Comparar que estrategia de administración de tiempo es mas productiva?
  - 24-6
  - 12-3
  - Free
- 

In [14]:
random.seed(RANDOM_SEED)

def simulate(n, met=(0,0), interruptions=True):
    for i in range(n):
        env = simpy.Environment()
        person = Person(env,'S', met, interruptions)
        env.run(until=200)
        print("Descanso: ",person.breaks)
        print("Tareas Completadas: ",person.completed_tasks)
        print("Duración Media de tareas: ",person.task_duration_sum/person.completed_tasks)
        print("Interrupciónes: ",person.interrupts)

simulate(1,met=(24.0,6.0),interruptions=False)

TypeError: simulate() missing 1 required positional argument: 'n'